# FGS Spelunker Quickstart

---
**Authors:** Derod Deal (dealderod@ufl.edu), Néstor Espinoza (nespinoza@stsci.edu) | **Last update:** July 24, 2023

**Program version:** ver. 1.0.1

The JWST telescope carries four different instruments: NIRCam, NIRSpec, MIRI and FGS/NIRISS --- the latter containing the Fine Guidance Sensor (FGS). FGS Spelunker is a package designed to conveniently analyze guidestar data. In this notebook, we cover the following main functions of this package.


1. [Getting started](#getting-started)
    - Installation
    - Using `fgs_splelunker`
2. [Downloading data](#downloading-data)
3. Spatial fitting guide stars
    - [Gaussian fitting](#gaussian-fitting)
    - [Quick fitting](#quick-fitting)
4. [Plotting parameters](#plotting-parameters)
5. Periodograms
    - [Creating a periodogram](#periodograms)
6. [Mnemonics](#mnemonics)
7. [Animations](#animations)
8. [Getting tables](#getting-tables)

# Getting started

To get started with FGS Spelunker, first call `spelunker.load` into a variable while setting a given Program ID.

In [ ]:
import os
import sys
os.chdir('/Users/ddeal/JWST-Treasure-Chest/')
sys.path.append('/Users/ddeal/JWST-FGS-Spelunker/JWST-FGS-spk-main/src/')

```python
import spelunker
spk = spelunker.load(pid=1534)
```

Calling load without the pid parameter `spelunker.load()` will initialize `spelunker` without downloading any of the files. This is useful if you already have timeseries arrays to work with.

## Downloading data

To load Spelunker with a given Program ID for JWST, simply call `spelunker.load` with the Program ID `pid` as a parameter. This will create a directory called `spelunker_results`, which is where the FITS files from a selected Program ID and other data will be downloaded and saved. You can define your own directory by using `dir=`.

> The Program IDs that can be loaded are limited to programs without an exclusive access period or are available publicly (https://jwst-docs.stsci.edu/accessing-jwst-data/jwst-data-retrieval/data-access-policy#DataAccessPolicy-Exclusiveaccessperiod). 

In [ ]:
import spelunker
spk = spelunker.load(dir='/Users/ddeal/JWST-Treasure-Chest/', pid='1534')

To download the data after initialization, use `spk.download()` with given proposal id with the optional parameters observation number `obs_num` and visit number `visit`. You can also set the calibration level `calib_level`. This information are required to use `astroquery.mast` to search and download the necessary files. The download function will download the selected files in the given directory and create a 2D array of the guidestar data as well as an array of time and a flux timeseries. The same parameters work with `spelunker.load`.

In [ ]:
spk2 = spelunker.load(pid=1534, obs_num='2', visit='1', calib_level=2)
spk2.download(1534, obs_num='2', visit='2', calib_level=2) # This overwrites the object data in spk2 with data from the input parameters

After we downloaded our data, we can access preprocessed spatial, time, and flux arrays for all FITS files images under the specified Program ID. Use the attributes `spk.fg_array`, `spk.fg_time`, and `spk.fg_flux` to access the arrays.

In [ ]:
spk2.fg_array.shape, spk2.fg_time.shape, spk2.fg_flux.shape

Previously downloaded FITS files in a given directory will not be re-downloaded. If there are multiple files downloaded for the given parameter, `spk.download` will automatically stitch the data from the files into an array based on the date and time for each file, along with the time and flux arrays. 

FGS Spelunker can also handle single fits files already stored locally by using:

```python
spk2.readfile('/Users/ddeal/Spelunker-older/JWST-Treasure-Chest-2023/mastDownload/JWST/jw01534001001_03101_00001_guider1/jw01534001001_gs-fg_2022340000825_cal.fits')
```

> Currently, `spk.readfile()` does not support the folling attributes: 
> - `fg_table`, 
> - `object_properties`.

## Spatial fitting guide stars

After downloading the data, we can perform spatial fitting gaussians to each frame in a guidestar timeseries. This uses parallel processing through `ray` to speed up the process. We can also perform quick fits to speed through a given timeseries, though this method is a lot less accurate in the fitting.

### Gaussian fitting

The downloaded data comes as a spatial timeseries of a selected guidestar. To measure the centriods and PSF width of each frame, we need to apply fitting. We will use Gaussian spatial fitting to measure x and y pixel coordinates, x and y standard deviations, thetha, and the offset. We will use the following derivation for fitting the guidestar frames: 

$$\large f(x,y) = A \exp \left(-\left(a(x-x_0)^2 + 2b(x-x_0)(y-y_0) + c(y-y_0)^2\right)\right) + \text{offset,}$$

where $\large a = \frac{\cos^2 \theta}{2 \sigma^2_X} + \frac{\sin^2 \theta}{2\sigma^2_Y} $, $ \large b = \frac{\sin 2\theta}{4 \sigma^2_X} + \frac{\sin 2\theta}{4\sigma^2_Y}$, and $ \large  c = \frac{\sin^2 \theta}{2 \sigma^2_X} + \frac{\cos^2 \theta}{2\sigma^2_Y}$ (https://en.wikipedia.org/wiki/Gaussian_function). To perform spatial gaussian fitting, use `gauss2d_fit` with guidestar arrays (the timeseries needs to be in an 8 by 8 array, which should be the same for all guidestar fine guidence products).

```python
spk.gauss2d_fit() # ncpus sets the number of cpu cores your computer has. Defaults to 4 cores.
```

In [ ]:
# We are going to limit the amount of frames that we input into gauss2d_fit and other methods
# since the gauss2d_fit can take a few houts for very large arrays.
spk.fg_array = spk.fg_array[0:10000]
spk.fg_flux = spk.fg_flux[0:10000]
spk.fg_time = spk.fg_time[0:10000]

In [ ]:
table_gauss_fit = spk.gauss2d_fit(ncpus=6) 

The `gauss2d_fit` function outputs an astropy table, which can bee accessed with the `spk.gaussfit_results` attribute. If `gauss2d_fit` fails to fit a frame, it will return `nan` for that frame.

In [ ]:
spk.gaussfit_results

### Quick fitting

There are some situations where you need to quickly obtain rough statistics of changes in guidestar products overtime. Quick fitting fits the x and y pixel locations and standard deviations as an astropy table using centroid and variance calculations. To perform quick fitting, run `quick_fit` with an appropriate array.

In [ ]:
table_quick_fit = spk.quick_fit()

In [ ]:
spk.quickfit_results

## Plotting parameters

We can plot a timeseries of a given parameter or flux from guidestars. The method `timeseries_binned_plot` will generate a matplotlib axes object of a given timeseries.

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize = (12,4), dpi=200)

ax = spk.timeseries_binned_plot()

Within guidestar data, changes in the PSF can impact the observed flux of the star. Certain events might see changes in all fitted parameters. In this case, subplots of each parameter will provide more information to the user about the event, giving them the change of guidestar position, brightness, and FWHM overtime. 

In [ ]:
ax = spk.timeseries_list_plot()

## Periodograms

FGS Spelunker comes with various tools to visualize and explore guidestar data. Periodograms are useful for guidestar products to detect periodicities not only within flux timeseries, but also within centroids, FWHM, theta, and offset. From a selected fitting method, we can use the table output to apply Lomb-Scargle periodograms to our parameters.

### `periodogram`

To obtain the power and frequencies of Lomb-Scargle periodograms for each fitted parameter, use `periodogram`. The periodograms for each given parameter from a fit can be conveniently plotted in a single figure with the same method.

In [ ]:
ax = spk.periodogram()

To get the frequency and power for each fitted parameter, use `spk.pgram_{parameter}`.
> Available parameters:
> - `spk.pgram_amplitude`
> - `spk.pgram_x_mean`
> - `spk.pgram_y_mean`
> - `spk.pgram_x_stddev`
> - `spk.pgram_y_stddev`
> - `spk.pgram_theta`
> - `spk.pgram_offset`

In [ ]:
freq = spk.pgram_x_mean[0] # periodogram frequency
power = spk.pgram_x_mean[1] # periodogram power

freq[0], power[0]

## Mnemonics

When observing the timeseries of the guidestar, there might be technical events from the JWST that causes changes in obtained data. For example, high gain antenna or filter changes in NIRCAM can cause noticeable changes in flux or other guidestar properties. We can overlay these events onto fitted parameters using `mnemonics` and `mnemonics_plot`. You will need a MAST API token to use `mnemonics`, as well as the `jwstuser` package.
- https://auth.mast.stsci.edu/docs/ (MAST API TOKEN)
- https://github.com/spacetelescope/jwstuser/tree/main (jwstuser)


> Current supported mnemonics: *SA_ZHGAUPST* (high-gain antenna), *INIS_FWMTRCURR* (NIRISS Filter Wheel Motor Current).


In [ ]:
spk2 = spelunker.load('/Users/ddeal/JWST-Treasure-Chest/', pid=1534)

In [ ]:
spk2.mast_api_token = 'enter_mast_token_id_here' # input mast_api token here!

fig, ax = plt.subplots(figsize=(12,4),dpi=200)

ax = spk2.mnemonics_local('GUIDESTAR')
ax = spk2.mnemonics('SA_ZHGAUPST', 60067.84, 60067.9)

ax.plot(spk2.fg_time, spk2.fg_flux)
plt.legend(loc=3)

plt.xlim(60067.84, 60067.9)

## Animations

Spatial data of guidestar imaging can bring essential information about how the point spread function changes overtime. Animations of the spatial timeseries are convenient and helpful methods to analyze guidestar data. To get a side by side comparison of the evolution of a spatial timeseries and a parameter, use `flux_spatial_timelapse_animation`.

> You may have to install `ffmpeg` on your computer to get `mp4` formats.

In [ ]:
plt.plot(spk2.fg_flux[2600:3100])

In [ ]:
spk2.flux_spatial_timelapse_animation(start=2600,stop=3100,) # to save an animation with a filename, use *filename=*. Defaults to movie.gif

## Getting tables

After downloading a selected proposal id with `download`, we can easily output metadata about each downloaded file, including extracted data from the filename including `visit_group`, `parallel_sequence_id`, and `exposure_number`. The guide star used in each file is also included, as well as filter magnitudes and other stellar properties.

In [ ]:
spk.fg_table # We can simply call this attribute after using spk.download() to obtain our table!

We can obtain a neat DataFrame of each tracked guidestar, which gives us information such as the intergation start times and galactic coordinates.

In [ ]:
spk.object_properties